## Message filter (from YAML spec)

This notebook demonstrates using Runtime Predicate to filter messages coming into an agent.

The `runtime_predicate` for an AgentSpec can be a `SimpleRuntimePredicate` with the expression being a JSONata filter expression.

The runtime predicate can be used to filter messages before the agent sees them. The filter cann be applied on -
- **message**: The dictionary for complete incoming message
- **agent_state**: The dictionary for the  current agent state
- **guild_state**: The dictionary for the current guild state

[ProbeAgent] --`{"payload":{"filter":true}}`-> (default_topic) **->** [Filtering Agent/FilteringAgent:wire_message] -> (accepted)

[ProbeAgent] --`{"payload":{"filter":false}}`-> (default_topic) **-X** [Filtering Agent/FilteringAgent:wire_message] X |NO MESSAGE|


In [ ]:
from pydantic import BaseModel


class FilteringMessage(BaseModel):
    """A message that can be filtered based on its content."""

    filter: bool
    content: str

In [ ]:
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder

# Create the Guild with the Routing Agent and the Routing Rules
guild = GuildBuilder.from_yaml_file("./004_message_filter.yaml").launch("myorg")

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.dsl import GuildTopics

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test the routing of messages.")
    .add_additional_topic("accepted")
    .add_additional_topic(GuildTopics.DEAD_LETTER_QUEUE)
    .build()
)

guild._add_local_agent(probe_agent)

In [ ]:
probe_agent.publish_with_guild_route(
    payload=FilteringMessage(filter=True, content="This is a test message"),
    topic=guild_default_topic,
)

In [ ]:
probe_agent.print_message_history()

In [ ]:
probe_agent.clear_messages()

probe_agent.publish_with_guild_route(
    payload=FilteringMessage(filter=False, content="This is message should be rejected"),
    topic=guild_default_topic,
)

In [ ]:
probe_agent.print_message_history()